In [35]:
import numpy
import pandas as pd
import folium
import requests
import bs4
import sklearn
import matplotlib
import dateparser
from bs4 import BeautifulSoup
from dateparser import parse

df = pd.read_csv("data.csv", names=['', 'How much', 'What', 'When', 'Where']);



def get_start_time(date_str):
    date_str = date_str.strip().strip('.')
    if '&' in date_str:
        date_str = date_str.split('&')[0]
    if '-' in date_str:
        date_str = date_str.split('-')[0].strip()
    if '.' in date_str:
        date_str = date_str.replace('.', ':')
    if date_str.startswith('Tues'):
        date_str = date_str.replace('Tues', 'Tue')
    if date_str.startswith('Thur'):
        date_str = date_str.replace('Thur', 'Thu')

    date = parse(date_str)
    return date


def start_time_to_float(b):
    try:
        return float("{}{:02d}".format(b.hour, b.minute))

    except:
        return numpy.math.nan



import re

def get_price(price_str):
    price_regexp = r"(?P<price>\d+)"
    if 'Free admission' in price_str:
        price = 0
    elif 'ratis' in price_str:
        price = 0
    else:
        m = re.search(price_regexp, price_str)
        try:
            price = int(m.group('price'))
        except:
            price = None
    return price



In [36]:
df.head()


,,How much,What,When,Where
0,NaN,How much,What,When,Where
1,0.0,NaN,Koncert med Rytmisk Center.,"Mon 20 November 2017, 6 pm - 10 pm.","Eliaskirken, Vesterbros Torv, Vesterbrogade 49..."
2,1.0,"(Entrance fee: 408,00 dkk - 508,00 dkk)",Five Finger Death Punch & In Flames.,"Mon 20 November 2017, 7 pm.","Royal Arena, Hannemanns AllÃ© 18-20, Copenhagen S"
3,2.0,NaN,Koncert med Lyngby Seniorkor.,"Mon 20 November 2017, 7 pm - 11 pm.","Sorgenfri Kirke og SognegÃ¥rd, Hummeltoften 1,..."
4,3.0,(Free admission),Mestre pÃ¥ Mandage IX.,"Mon 20 November 2017, 7.30 pm.","Sankt Markus Kirke, Forchhammersvej 12A, Frede..."


In [37]:
df['When'] = [start_time_to_float(get_start_time(date)) for date in df['When']]


In [41]:
df["How much"] = [ get_price(str(price_str)) for price_str in df["How much"] ]

In [43]:
df.head()

,,How much,What,When,Where
0,NaN,NaN,What,NaN,Where
1,0.0,NaN,Koncert med Rytmisk Center.,1800.0,"Eliaskirken, Vesterbros Torv, Vesterbrogade 49..."
2,1.0,408.0,Five Finger Death Punch & In Flames.,1900.0,"Royal Arena, Hannemanns AllÃ© 18-20, Copenhagen S"
3,2.0,NaN,Koncert med Lyngby Seniorkor.,1900.0,"Sorgenfri Kirke og SognegÃ¥rd, Hummeltoften 1,..."
4,3.0,0.0,Mestre pÃ¥ Mandage IX.,1930.0,"Sankt Markus Kirke, Forchhammersvej 12A, Frede..."


In [64]:
import json
geo = json.load(open('locs2.json'))


In [102]:
dict = {}
for keys, value in geo.items():
    for key in keys.split(","):
         dict[key] = value
for keys, value in geo.items():
    dict[keys] = value

In [103]:
dict

{' 1. tv.': [None, None],
 ' 1.tv.': [55.6741128, 12.543863],
 ' 3. sal': [55.67636659999999, 12.5747082],
 ' Ahlmanns Allé 6': [55.73745899999999, 12.573438],
 ' Albertslund': [55.6847112, 12.3700475],
 ' Albertslund Rådhus': [55.65691899999999, 12.3578431],
 ' Albjergparken 4': [None, None],
 ' Alhambravej 1': [55.6741128, 12.543863],
 ' Amagerbrogade 71': [55.6608674, 12.6053029],
 ' Andebakkesti 5': [55.6783771, 12.527761],
 ' Axeltorv 1': [55.6754392, 12.5644863],
 ' Axeltorv 3': [55.6756414, 12.5642281],
 ' Bagsværd': [55.76075609999999, 12.4568021],
 ' Bagsværd Hovedgade 116 P': [55.76075609999999, 12.4568021],
 ' Ballerup': [55.730546, 12.35193],
 ' Ballerup Idrætsby 4': [55.72161, 12.36926],
 ' Baltorpvej 20': [55.730546, 12.35193],
 ' Bellahøjvej 20': [55.701843, 12.511142],
 ' Bernstorffsvej 73': [55.736265, 12.5554159],
 ' Bibliotekstorvet 1-3': [55.6568846, 12.3557788],
 ' Bjelkes Allé 19-21': [55.6929642, 12.5442185],
 ' Blegdamsvej 1 A': [55.691804, 12.561817],
 ' Blågår

In [108]:
df["loc"] = [ dict[a.split(',')[1]].strip('[').strip(']').slit(',') if a in dict else "" for a in df['Where']]

SyntaxError: invalid syntax (<ipython-input-108-cb06a86fb605>, line 1)

In [106]:
df

,,How much,What,When,Where,loc
0,NaN,NaN,What,NaN,Where,
1,0.0,NaN,Koncert med Rytmisk Center.,1800.0,"Eliaskirken, Vesterbros Torv, Vesterbrogade 49...","[55.671907, 12.5552659]"
2,1.0,408.0,Five Finger Death Punch & In Flames.,1900.0,"Royal Arena, Hannemanns AllÃ© 18-20, Copenhagen S",
3,2.0,NaN,Koncert med Lyngby Seniorkor.,1900.0,"Sorgenfri Kirke og SognegÃ¥rd, Hummeltoften 1,...",
4,3.0,0.0,Mestre pÃ¥ Mandage IX.,1930.0,"Sankt Markus Kirke, Forchhammersvej 12A, Frede...","[55.6798977, 12.5526499]"
5,4.0,100.0,Moderne mandag - Beates klaverbog.,1930.0,"Metronomen, GodthÃ¥bsvej 33, Frederiksberg",
6,5.0,95.0,Gaffa Session: SÃ¸ren Huss.,1930.0,"KU.BE, Dirch Passers AllÃ© 4, Frederiksberg",
7,6.0,325.0,SAGA - 40th Anniversary - The Final Chapter To...,2000.0,"Amager Bio, Ãresundsvej 6, Copenhagen S",
8,7.0,325.0,The Kooks (uk) - Venteliste.,2000.0,"VEGA, Enghavevej 40, Copenhagen V","[55.6680687, 12.5444302]"
9,8.0,80.0,The Pagsberg Bigband.,2000.0,"K-I-B, Kulturhuset Islands Brygge, Islands Bry...","[55.666846, 12.5762349]"


In [107]:
df.to_csv('data_loc_1.2.csv')